# Pandas SpeedUp

In [1]:
import pandas as pd
import numpy as np

## Task 1:
#### Drop dollar and convert to integer

In [2]:
def make_s(n_rows):
    tmp = pd.DataFrame({'price': (100*np.random.rand(n_rows)).astype(int), 
                        'feature': np.zeros(n_rows)})
    tmp['price'] = tmp['price'].astype(str) + '$'
    return tmp

data = make_s(5)
data

,price,feature
0,25$,0.0
1,61$,0.0
2,61$,0.0
3,71$,0.0
4,61$,0.0


In [3]:
data['price($)_v1'] = data['price'].apply(lambda x: int(x[:-1]))
data['price($)_v2'] = data['price'].apply(lambda x: x[:-1]).astype(int)
data['price($)_v3'] = data['price'].apply(lambda x: x.replace('$', '')).astype(int)
data['price($)_v4'] = data['price'].str.replace('$', '').astype(int)
# My solution:
data['price($)_v5'] = np.array(''.join(data['price'].values)[:-1].split('$'), dtype=int)

data

,price,feature,price($)_v1,price($)_v2,price($)_v3,price($)_v4,price($)_v5
0,25$,0.0,25,25,25,25,25
1,61$,0.0,61,61,61,61,61
2,61$,0.0,61,61,61,61,61
3,71$,0.0,71,71,71,71,71
4,61$,0.0,61,61,61,61,61


In [4]:
data = make_s(10000000)

In [5]:
%timeit data['price($)_v1'] = data['price'].apply(lambda x: int(x[:-1]))
%timeit data['price($)_v2'] = data['price'].apply(lambda x: x[:-1]).astype(int)
%timeit data['price($)_v3'] = data['price'].apply(lambda x: x.replace('$', '')).astype(int)
%timeit data['price($)_v4'] = data['price'].str.replace('$', '').astype(int)
# My solution:
%timeit data['price($)_v5'] = np.array(''.join(data['price'].values)[:-1].split('$'), dtype=int)

5.85 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.59 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.63 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.85 s ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.72 s ± 77.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Task 2:
#### Binarize

In [6]:
def make_t(n_rows):
    tmp = pd.DataFrame({'type': np.where(np.random.rand(n_rows)<0.5, 'A', 'B'), 'feature': np.zeros(n_rows)})
    return tmp

data = make_t(5)
data

,type,feature
0,B,0.0
1,A,0.0
2,B,0.0
3,B,0.0
4,A,0.0


In [7]:
from sklearn import preprocessing

data['type_v1'] = data['type'].apply(lambda x: 1 if x == "A" else 0)
data['type_v2'] = (data['type']=='A').astype(int)
data['type_v3'] = np.where(data['type'] == 'A', 1 ,0)
data['type_v4'] = data['type'].map({'A': 1, 'B': 0})
data['type_v5'] = data['type'].factorize()[0] 
data['type_v6'] = pd.get_dummies(data['type'])['A'] 
data['type_v7'] = preprocessing.LabelEncoder().fit_transform(data['type']) 
# My solution:
data['type_v8'] = (data['type'].values == 'A').astype(int)

data

,type,feature,type_v1,type_v2,type_v3,type_v4,type_v5,type_v6,type_v7,type_v8
0,B,0.0,0,0,0,0,0,0,1,0
1,A,0.0,1,1,1,1,1,1,0,1
2,B,0.0,0,0,0,0,0,0,1,0
3,B,0.0,0,0,0,0,0,0,1,0
4,A,0.0,1,1,1,1,1,1,0,1


In [8]:
data = make_t(10000000)

In [9]:
%timeit data['type_v1'] = data['type'].apply(lambda x: 1 if x == "A" else 0)
%timeit data['type_v2'] = (data['type']=='A').astype(int)
%timeit data['type_v3'] = np.where(data['type'] == 'A', 1 ,0)
%timeit data['type_v4'] = data['type'].map({'A': 1, 'B': 0})
%timeit data['type_v5'] = data['type'].factorize()[0] 
%timeit data['type_v6'] = pd.get_dummies(data['type'])['A'] 
%timeit data['type_v7'] = preprocessing.LabelEncoder().fit_transform(data['type']) 
# My solution:
%timeit data['type_v8'] = (data['type'].values == 'A').astype(int)

3.09 s ± 57.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
485 ms ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
517 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
510 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
352 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
438 ms ± 34.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.51 s ± 27.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
236 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Task 3:
#### A/B Split

In [10]:
def make_ab(n_rows):
    tmp = pd.DataFrame({'A': (100*np.random.rand(n_rows)).astype(int), 'B': (100*np.random.rand(n_rows)).astype(int), 'feature': np.zeros(n_rows)})
    tmp['A/B'] = tmp['A'].astype(str) + '/' + tmp['B'].astype(str)
    del tmp['A']
    del tmp['B']
    return tmp

data = make_ab(5)
data

,feature,A/B
0,0.0,63/70
1,0.0,91/60
2,0.0,61/39
3,0.0,38/76
4,0.0,34/70


In [11]:
def solve_ab_1(data):
    tmp = data['A/B'].str.split('/')
    data['A_v1'] = tmp.apply(lambda x: x[0])
    data['B_v1'] = tmp.apply(lambda x: x[1])
    
def solve_ab_2(data):
    data[['A_v2', 'B_v2']] = pd.DataFrame(data['A/B'].str.split('/', 1).tolist())
    
def solve_ab_3(data):
    data[['A_v3', 'B_v3']] = data['A/B'].str.split('/', expand=True)

def solve_ab_4(data):
    st = '/'.join(data['A/B'])
    data[['A_v4', 'B_v4']] = pd.DataFrame(np.array(st.split('/')).reshape(-1, 2))

def solve_ab_5(data):
    arr = np.array('/'.join(data['A/B'].values).split('/')).reshape(-1, 2)
    data['A_v5'] = arr[:, 0]
    data['B_v5'] = arr[:, 1]
    
solve_ab_1(data)
solve_ab_2(data)
solve_ab_3(data)
solve_ab_4(data)
# My solution:
solve_ab_5(data)

data

,feature,A/B,A_v1,B_v1,A_v2,B_v2,A_v3,B_v3,A_v4,B_v4,A_v5,B_v5
0,0.0,63/70,63,70,63,70,63,70,63,70,63,70
1,0.0,91/60,91,60,91,60,91,60,91,60,91,60
2,0.0,61/39,61,39,61,39,61,39,61,39,61,39
3,0.0,38/76,38,76,38,76,38,76,38,76,38,76
4,0.0,34/70,34,70,34,70,34,70,34,70,34,70


In [12]:
data = make_ab(1000000)

In [13]:
%timeit solve_ab_1(data)
%timeit solve_ab_2(data)
%timeit solve_ab_3(data)
%timeit solve_ab_4(data)
# My solution:
%timeit solve_ab_5(data)

985 ms ± 50.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
953 ms ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.17 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
679 ms ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
511 ms ± 34.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Task 4:
#### Fill NaN's with mean

In [14]:
def make_t(n_rows):
    tmp = pd.DataFrame({'type': np.where(np.random.rand(n_rows)<0.5, 'train', 'test'),
                        'feature': np.where(np.random.rand(n_rows)<0.5, (100*np.random.rand(n_rows)).astype(int), np.nan)})
    tmp['feature_v1'] = tmp['feature']
    tmp['feature_v2'] = tmp['feature']
    tmp['feature_v3'] = tmp['feature']
    tmp['feature_v4'] = tmp['feature']
    tmp['feature_v5'] = tmp['feature']
    return tmp

data = make_t(20)
data

,type,feature,feature_v1,feature_v2,feature_v3,feature_v4,feature_v5
0,train,NaN,NaN,NaN,NaN,NaN,NaN
1,train,72.0,72.0,72.0,72.0,72.0,72.0
2,test,17.0,17.0,17.0,17.0,17.0,17.0
3,train,63.0,63.0,63.0,63.0,63.0,63.0
4,train,NaN,NaN,NaN,NaN,NaN,NaN
5,test,NaN,NaN,NaN,NaN,NaN,NaN
6,test,NaN,NaN,NaN,NaN,NaN,NaN
7,test,95.0,95.0,95.0,95.0,95.0,95.0
8,test,NaN,NaN,NaN,NaN,NaN,NaN
9,test,15.0,15.0,15.0,15.0,15.0,15.0


In [15]:
def solve_nan_1(data):
    name = 'feature_v1'
    data.loc[data['type'] == 'test', name] = \
        data[data['type'] == 'test'][name].fillna(data[data['type'] == 'test'][name].mean())
    data.loc[data['type'] == 'train', name] = \
        data[data['type'] == 'train'][name].fillna(data[data['type'] == 'train'][name].mean())

def solve_nan_2(data):
    name = 'feature_v2'
    data[name] = data.groupby('type')[name].transform(lambda x: x.fillna(x.mean()))

def solve_nan_3(data):
    name = 'feature_v3'
    data.loc[data[name].isnull(), name] = data.groupby('type')[name].transform('mean')

def solve_nan_4(data):
    name = 'feature_v4'
    data[name] = np.where(data[name].isnull(), data['type'].map(data.groupby('type')[name].mean()), data[name])

def solve_nan_5(data):
    name = 'feature_v5'
    data[name].fillna(data[name].groupby(data['type']).transform(np.mean), inplace=True)
    
solve_nan_1(data)
solve_nan_2(data)
solve_nan_3(data)
solve_nan_4(data)
# My solution:
solve_nan_5(data)
    
data

,type,feature,feature_v1,feature_v2,feature_v3,feature_v4,feature_v5
0,train,NaN,47.25,47.25,47.25,47.25,47.25
1,train,72.0,72.00,72.00,72.00,72.00,72.00
2,test,17.0,17.00,17.00,17.00,17.00,17.00
3,train,63.0,63.00,63.00,63.00,63.00,63.00
4,train,NaN,47.25,47.25,47.25,47.25,47.25
5,test,NaN,45.00,45.00,45.00,45.00,45.00
6,test,NaN,45.00,45.00,45.00,45.00,45.00
7,test,95.0,95.00,95.00,95.00,95.00,95.00
8,test,NaN,45.00,45.00,45.00,45.00,45.00
9,test,15.0,15.00,15.00,15.00,15.00,15.00


In [16]:
data = make_t(10000000)

In [17]:
%timeit solve_nan_1(data)
%timeit solve_nan_2(data)
%timeit solve_nan_3(data)
%timeit solve_nan_4(data)
# My solution:
%timeit solve_nan_5(data)

5.01 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.66 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
820 ms ± 84.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.41 s ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
668 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
